## Corpus testing

This notebook takes works with known labels and looks how good it is attributed to its author. 

In [1]:
import pandas as pd
import numpy as np

import re

from sklearn.preprocessing import StandardScaler

import logging

logging.basicConfig(level="INFO")
logger = logging.getLogger("ruzicka")

from ruzicka.BDIVerifier import BDIVerifier

### I. Authors to themselves

In [2]:
# load preprocessed data

corpus = pd.read_csv('03_tests/authors_themselves/to-test_2000_200mfw_rfreq.csv') # rel freq

corpus

,work,author,chunk_num,tag,de,la,les,vous,il,l,...,effet,mieux,donner,jamais,enfin,voir,rendre,ici,beaucoup,cependant
0,Avis,Baudeau,0,0__Baudeau_Avis,3.45,1.40,2.35,0.65,1.25,0.65,...,0.05,0.00,0.05,0.00,0.20,0.00,0.00,0.05,0.05,0.00
1,Eclaircissemens,Baudeau,0,0__Baudeau_Eclaircissemens,4.40,2.60,2.40,1.65,0.85,2.20,...,0.25,0.00,0.05,0.10,0.00,0.00,0.00,0.00,0.00,0.00
2,Explication,Baudeau,0,0__Baudeau_Explication,4.90,3.10,4.65,0.95,1.40,1.15,...,0.00,0.00,0.05,0.00,0.05,0.05,0.00,0.10,0.05,0.00
3,Idees sur les besoins 1,Baudeau,0,0__Baudeau_Idees sur les besoins 1,4.65,3.00,3.20,0.10,0.80,2.00,...,0.00,0.10,0.00,0.20,0.00,0.05,0.00,0.10,0.00,0.00
4,Idees sur les besoins 2,Baudeau,0,0__Baudeau_Idees sur les besoins 2,4.65,3.00,3.35,0.05,0.80,2.10,...,0.00,0.10,0.00,0.20,0.00,0.05,0.00,0.10,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2692,elements T3,Marmontel,94,94__Marmontel_elements T3,5.45,3.70,1.60,0.00,1.10,3.05,...,0.00,0.00,0.15,0.20,0.00,0.05,0.05,0.00,0.10,0.05
2693,elements T1,Marmontel,95,95__Marmontel_elements T1,4.35,3.25,0.95,0.10,2.05,3.45,...,0.05,0.05,0.05,0.05,0.05,0.20,0.35,0.00,0.00,0.05
2694,elements T3,Marmontel,95,95__Marmontel_elements T3,4.20,3.10,2.95,0.10,1.65,2.65,...,0.00,0.00,0.10,0.15,0.00,0.20,0.05,0.05,0.05,0.05
2695,elements T3,Marmontel,96,96__Marmontel_elements T3,4.15,3.00,1.15,0.00,1.45,2.75,...,0.05,0.05,0.05,0.05,0.05,0.00,0.10,0.00,0.00,0.00


In [3]:
raw_df = corpus.iloc[:, :4] # leaves metadata
X = corpus.iloc[:, 4:] # leaves only word columns

raw_df

,work,author,chunk_num,tag
0,Avis,Baudeau,0,0__Baudeau_Avis
1,Eclaircissemens,Baudeau,0,0__Baudeau_Eclaircissemens
2,Explication,Baudeau,0,0__Baudeau_Explication
3,Idees sur les besoins 1,Baudeau,0,0__Baudeau_Idees sur les besoins 1
4,Idees sur les besoins 2,Baudeau,0,0__Baudeau_Idees sur les besoins 2
...,...,...,...,...
2692,elements T3,Marmontel,94,94__Marmontel_elements T3
2693,elements T1,Marmontel,95,95__Marmontel_elements T1
2694,elements T3,Marmontel,95,95__Marmontel_elements T3
2695,elements T3,Marmontel,96,96__Marmontel_elements T3


In [4]:
# numer of rows and columns in the X (word frequencies subset)
X.shape

(2697, 200)

In [5]:
# create numeric author labels
labels, label_uniques = raw_df.author.factorize()
raw_df.insert(1, "author_label", labels)
raw_df

,work,author_label,author,chunk_num,tag
0,Avis,0,Baudeau,0,0__Baudeau_Avis
1,Eclaircissemens,0,Baudeau,0,0__Baudeau_Eclaircissemens
2,Explication,0,Baudeau,0,0__Baudeau_Explication
3,Idees sur les besoins 1,0,Baudeau,0,0__Baudeau_Idees sur les besoins 1
4,Idees sur les besoins 2,0,Baudeau,0,0__Baudeau_Idees sur les besoins 2
...,...,...,...,...,...
2692,elements T3,10,Marmontel,94,94__Marmontel_elements T3
2693,elements T1,10,Marmontel,95,95__Marmontel_elements T1
2694,elements T3,10,Marmontel,95,95__Marmontel_elements T3
2695,elements T3,10,Marmontel,96,96__Marmontel_elements T3


In [6]:
# check unique authors
label_uniques.values

array(['Baudeau', 'Chastellux', 'Condorcet', 'Deleyre', 'dHolbach',
       'Diderot', 'Guibert', 'Jaucourt', 'Jussieu', 'La Grange',
       'Marmontel', 'Meister', 'Morellet', 'Naigeon', 'Pechmeja',
       'Raynal', 'Rivière', 'Saint-Lambert'], dtype=object)

In [7]:
# set verifier
rng = np.random.default_rng(42)

bdi_mm = BDIVerifier(
    metric='minmax', nb_bootstrap_iter=1000, rnd_prop=0.35, random_state=rng
)

In [9]:
# Loop that takes a work from each author
# and looks whether its (known) author is a good much according to BDI;
# nb: not getting results for a work vs all authors
# (writing results in a .csv, visualisation done later with R)

# select an author, look for his unique works
for a in label_uniques.values:
    #print(author)
    
    # take a subset with only works by an author, find unique works
    unique_works = raw_df[raw_df.author == a]['work'].unique()
    # unique_works is a numpy array of works of each author
    
    for w in unique_works:
        print(f"\n\n#######################\n{a}: {w}\n")
        
        # select each work as a problem set (problems-metadata, problems_X-word freqs)
        problems = raw_df[raw_df.work == w].reset_index(drop=True).copy()
        problems_X = X[raw_df.work == w].reset_index(drop=True).copy()
        print(problems.iloc[1:2,:])
        
        # extract the rest of the corpus
        rest = raw_df[raw_df.work != w].reset_index(drop=True).copy()
        rest_X = X[raw_df.work != w].reset_index(drop=True).copy()
        #print(f"\nRest of the corpus:\n{rest.iloc[1:6,:]}")
        
        # scaling
        ss = StandardScaler(with_mean=False).fit(rest_X)
        rest_scaled_X = ss.transform(rest_X)
        problems_scaled_X = ss.transform(problems_X)
        
        #print(problems_scaled_X)
        
        # fit 
        bdi_mm.fit(rest_scaled_X, rest.author_label)
        
        # prediction
        bdi_mm.predict_proba(
            problems_scaled_X, 
            # get_loc - select the author of the work (? is that right??)
            [label_uniques.get_loc(a)] * problems_scaled_X.shape[0]
        )
        
        # output results
        res = pd.DataFrame(dict(zip(problems.tag, bdi_mm._dist_arrays)))
        
        # writing
        fh = '03_tests/authors_themselves/bdi_res/'+a+'_'+w+'.csv'
        res.to_csv(fh)
        print('Results written in:',fh)
        
        rest_scaled_X = None
        res = None

03/18/2025 01:23:23 [ruzicka:INFO] Fitting on 2672 documents...
03/18/2025 01:23:23 [ruzicka:INFO] Predicting on 25 documents




#######################
Baudeau: Avis 

    work  author_label   author  chunk_num               tag
1  Avis              0  Baudeau          1  1__Baudeau_Avis 


03/18/2025 01:23:27 [ruzicka:INFO] Fitting on 2678 documents...
03/18/2025 01:23:27 [ruzicka:INFO] Predicting on 19 documents


Results written in: 03_tests/authors_themselves/bdi_res/Baudeau_Avis .csv


#######################
Baudeau: Eclaircissemens

              work  author_label   author  chunk_num  \
1  Eclaircissemens             0  Baudeau          1   

                          tag  
1  1__Baudeau_Eclaircissemens  


03/18/2025 01:23:31 [ruzicka:INFO] Fitting on 2686 documents...
03/18/2025 01:23:31 [ruzicka:INFO] Predicting on 11 documents


...

Results written in: 03_tests/authors_themselves/bdi_res/Saint-Lambert_Oeuvres Phil T2.csv


#######################
Saint-Lambert: Oeuvres Phil T3

              work  author_label         author  chunk_num  \
1  Oeuvres Phil T3            17  Saint-Lambert          1   

                                tag  
1  1__Saint-Lambert_Oeuvres Phil T3  


03/18/2025 01:33:10 [ruzicka:INFO] Fitting on 2652 documents...
03/18/2025 01:33:10 [ruzicka:INFO] Predicting on 45 documents


Results written in: 03_tests/authors_themselves/bdi_res/Saint-Lambert_Oeuvres Phil T3.csv


#######################
Saint-Lambert: Oeuvres Phil T4

              work  author_label         author  chunk_num  \
1  Oeuvres Phil T4            17  Saint-Lambert          1   

                                tag  
1  1__Saint-Lambert_Oeuvres Phil T4  


03/18/2025 01:33:19 [ruzicka:INFO] Fitting on 2675 documents...
03/18/2025 01:33:19 [ruzicka:INFO] Predicting on 22 documents


Results written in: 03_tests/authors_themselves/bdi_res/Saint-Lambert_Oeuvres Phil T4.csv


#######################
Saint-Lambert: Oeuvres Phil T5

              work  author_label         author  chunk_num  \
1  Oeuvres Phil T5            17  Saint-Lambert          1   

                                tag  
1  1__Saint-Lambert_Oeuvres Phil T5  
Results written in: 03_tests/authors_themselves/bdi_res/Saint-Lambert_Oeuvres Phil T5.csv


# FP1

Test FP1 chunks with BDI

In [52]:
# load preprocessed data

corpus = pd.read_csv('03_tests/fp1/2000_words/FP1_2000_200mfw_rfreq.csv') # rel freq

corpus

,work,author,chunk_num,tag,de,des,et,la,les,vous,...,voit,rendre,beaucoup,seul,ici,mieux,donner,enfin,voir,cependant
0,Avis,Baudeau,0,0__Baudeau_Avis,3.45,2.20,2.50,1.40,2.35,0.65,...,0.05,0.00,0.05,0.05,0.05,0.00,0.05,0.20,0.00,0.00
1,Eclaircissemens,Baudeau,0,0__Baudeau_Eclaircissemens,4.40,2.85,2.80,2.60,2.40,1.65,...,0.05,0.00,0.00,0.05,0.00,0.00,0.05,0.00,0.00,0.00
2,Explication,Baudeau,0,0__Baudeau_Explication,4.90,2.60,3.80,3.10,4.65,0.95,...,0.00,0.00,0.05,0.10,0.10,0.00,0.05,0.05,0.05,0.00
3,Idees sur l-administration,Baudeau,0,0__Baudeau_Idees sur l-administration,5.60,1.90,2.20,3.65,1.85,0.10,...,0.00,0.00,0.10,0.10,0.00,0.05,0.05,0.10,0.00,0.05
4,Idees sur la puissance,Baudeau,0,0__Baudeau_Idees sur la puissance,4.90,1.85,2.50,2.35,2.75,0.10,...,0.00,0.10,0.05,0.00,0.00,0.10,0.00,0.10,0.05,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4429,elements T3,Marmontel,96,96__Marmontel_elements T3,4.15,1.20,3.35,3.00,1.15,0.00,...,0.05,0.10,0.00,0.25,0.00,0.05,0.05,0.05,0.00,0.00
4430,Histoire Generale T19,Deleyre,97,97__Deleyre_Histoire Generale T19,5.25,1.85,0.75,3.75,2.65,0.00,...,0.00,0.05,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4431,elements T3,Marmontel,97,97__Marmontel_elements T3,4.60,0.95,3.10,3.40,1.55,0.00,...,0.00,0.10,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00
4432,Histoire Generale T19,Deleyre,98,98__Deleyre_Histoire Generale T19,4.90,2.95,0.75,3.25,3.20,0.05,...,0.05,0.00,0.20,0.00,0.00,0.05,0.00,0.00,0.05,0.05


In [53]:
raw_df = corpus.iloc[:, :4] # leaves metadata
X = corpus.iloc[:, 4:] # leaves only word columns

raw_df

,work,author,chunk_num,tag
0,Avis,Baudeau,0,0__Baudeau_Avis
1,Eclaircissemens,Baudeau,0,0__Baudeau_Eclaircissemens
2,Explication,Baudeau,0,0__Baudeau_Explication
3,Idees sur l-administration,Baudeau,0,0__Baudeau_Idees sur l-administration
4,Idees sur la puissance,Baudeau,0,0__Baudeau_Idees sur la puissance
...,...,...,...,...
4429,elements T3,Marmontel,96,96__Marmontel_elements T3
4430,Histoire Generale T19,Deleyre,97,97__Deleyre_Histoire Generale T19
4431,elements T3,Marmontel,97,97__Marmontel_elements T3
4432,Histoire Generale T19,Deleyre,98,98__Deleyre_Histoire Generale T19


In [54]:
# numer of rows and columns in the X (word frequencies subset)
X.shape

(4434, 200)

In [55]:
# create numeric author labels
labels, label_uniques = raw_df.author.factorize()
raw_df.insert(1, "author_label", labels)
raw_df

,work,author_label,author,chunk_num,tag
0,Avis,0,Baudeau,0,0__Baudeau_Avis
1,Eclaircissemens,0,Baudeau,0,0__Baudeau_Eclaircissemens
2,Explication,0,Baudeau,0,0__Baudeau_Explication
3,Idees sur l-administration,0,Baudeau,0,0__Baudeau_Idees sur l-administration
4,Idees sur la puissance,0,Baudeau,0,0__Baudeau_Idees sur la puissance
...,...,...,...,...,...
4429,elements T3,11,Marmontel,96,96__Marmontel_elements T3
4430,Histoire Generale T19,3,Deleyre,97,97__Deleyre_Histoire Generale T19
4431,elements T3,11,Marmontel,97,97__Marmontel_elements T3
4432,Histoire Generale T19,3,Deleyre,98,98__Deleyre_Histoire Generale T19


In [56]:
# select an "unknown" work

problems = raw_df[raw_df.work == "FP1"].reset_index(drop=True).copy()
problems_X = X[raw_df.work == "FP1"].reset_index(drop=True).copy()
problems

,work,author_label,author,chunk_num,tag
0,FP1,7,HDI,0,0__HDI_FP1
1,FP1,7,HDI,1,1__HDI_FP1
2,FP1,7,HDI,2,2__HDI_FP1


In [57]:
# extract the rest of the corpus

rest = raw_df[raw_df.work != "FP1"].reset_index(drop=True).copy()
rest_X = X[raw_df.work != "FP1"].reset_index(drop = True).copy()
rest

,work,author_label,author,chunk_num,tag
0,Avis,0,Baudeau,0,0__Baudeau_Avis
1,Eclaircissemens,0,Baudeau,0,0__Baudeau_Eclaircissemens
2,Explication,0,Baudeau,0,0__Baudeau_Explication
3,Idees sur l-administration,0,Baudeau,0,0__Baudeau_Idees sur l-administration
4,Idees sur la puissance,0,Baudeau,0,0__Baudeau_Idees sur la puissance
...,...,...,...,...,...
4426,elements T3,11,Marmontel,96,96__Marmontel_elements T3
4427,Histoire Generale T19,3,Deleyre,97,97__Deleyre_Histoire Generale T19
4428,elements T3,11,Marmontel,97,97__Marmontel_elements T3
4429,Histoire Generale T19,3,Deleyre,98,98__Deleyre_Histoire Generale T19


**Scaling**

In [58]:
ss = StandardScaler(with_mean=False).fit(rest_X)

In [59]:
rest_scaled_X = ss.transform(rest_X)
problems_scaled_X = ss.transform(problems_X)

**Verification**

In [60]:
# set verifier
rng = np.random.default_rng(42)

bdi_mm = BDIVerifier(
    metric='minmax', nb_bootstrap_iter=1000, rnd_prop=0.35, random_state=rng
)

In [61]:
# fit 
bdi_mm.fit(rest_scaled_X, rest.author_label)

03/18/2025 02:31:31 [ruzicka:INFO] Fitting on 4431 documents...


In [62]:
# check unique authors
label_uniques.values

array(['Baudeau', 'Chastellux', 'Condorcet', 'Deleyre', 'dHolbach',
       'Diderot', 'Guibert', 'HDI', 'Jaucourt', 'Jussieu', 'La Grange',
       'Marmontel', 'Meister', 'Morellet', 'Naigeon', 'Pechmeja',
       'Raynal', 'Rivière', 'Saint-Lambert'], dtype=object)

In [63]:
for label in label_uniques.values:
    print(f"Testing against {label}")
    code = label_uniques.get_loc(label)
    print(
        f"Bootstrap Match Strength (one per chunk, 0-1.0): {bdi_mm.predict_proba(problems_scaled_X, [code] * problems_scaled_X.shape[0])}"
    )

03/18/2025 02:31:35 [ruzicka:INFO] Predicting on 3 documents


Testing against Baudeau


03/18/2025 02:31:36 [ruzicka:INFO] Predicting on 3 documents


Bootstrap Match Strength (one per chunk, 0-1.0): [0.656 0.717 0.722]
Testing against Chastellux


03/18/2025 02:31:36 [ruzicka:INFO] Predicting on 3 documents


...

Bootstrap Match Strength (one per chunk, 0-1.0): [0.216 0.555 0.605]
Testing against Raynal


03/18/2025 02:31:50 [ruzicka:INFO] Predicting on 3 documents


Bootstrap Match Strength (one per chunk, 0-1.0): [0.421 0.559 0.657]
Testing against Rivière


03/18/2025 02:31:50 [ruzicka:INFO] Predicting on 3 documents


Bootstrap Match Strength (one per chunk, 0-1.0): [0.572 0.51  0.407]
Testing against Saint-Lambert
Bootstrap Match Strength (one per chunk, 0-1.0): [0.689 0.92  0.89 ]


In [64]:
authors = label_uniques.values

fh = ''

for a in authors:

    print(a)
    
    bdi_mm.predict_proba(
        problems_scaled_X, [label_uniques.get_loc(a)] * problems_scaled_X.shape[0]
    )

    
    x = pd.DataFrame(dict(zip(problems.tag, bdi_mm._dist_arrays)))
    
    fh = '03_tests/fp1/2000_words/mfw200/fp1_vs_' + a + '.csv'

    x.to_csv(fh)

03/18/2025 02:31:57 [ruzicka:INFO] Predicting on 3 documents


Baudeau


03/18/2025 02:31:57 [ruzicka:INFO] Predicting on 3 documents


Chastellux


03/18/2025 02:31:58 [ruzicka:INFO] Predicting on 3 documents


...

Raynal


03/18/2025 02:32:12 [ruzicka:INFO] Predicting on 3 documents


Rivière


03/18/2025 02:32:12 [ruzicka:INFO] Predicting on 3 documents


Saint-Lambert


# III. Work vs all authors
Here random 2 works from each author are used to see how good BDI will recognise true author (same settings as in the actual analysis)

In [9]:
# load preprocessed data

corpus = pd.read_csv('03_tests/authors_vs_all/5-authors_5works_2k_200mfw_rfreq.csv') # rel freq

corpus

,work,author,chunk_num,tag,de,la,les,des,et,l,...,fort,encore,autant,seul,plusieurs,assez,enfin,presque,beaucoup,donner
0,Eclaircissemens,Baudeau,0,0__Baudeau_Eclaircissemens,4.40,2.60,2.40,2.85,2.80,2.20,...,0.10,0.05,0.05,0.05,0.10,0.00,0.00,0.05,0.00,0.05
1,Explication,Baudeau,0,0__Baudeau_Explication,4.90,3.10,4.65,2.60,3.80,1.15,...,0.05,0.10,0.00,0.10,0.20,0.10,0.05,0.05,0.05,0.05
2,Idees sur les besoins 2,Baudeau,0,0__Baudeau_Idees sur les besoins 2,2.00,1.25,1.50,2.15,1.15,1.00,...,0.00,0.05,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00
3,Lettres a un magistraTtxt,Baudeau,0,0__Baudeau_Lettres a un magistraTtxt,4.15,3.60,4.00,2.55,3.20,1.00,...,0.05,0.25,0.05,0.20,0.05,0.10,0.05,0.00,0.10,0.00
4,Resultats,Baudeau,0,0__Baudeau_Resultats,4.60,2.45,2.95,3.30,3.05,2.20,...,0.00,0.15,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,Anecdotes historiques 1,Raynal,9,9__Raynal_Anecdotes historiques 1,4.40,3.10,3.35,1.15,3.15,1.20,...,0.15,0.10,0.05,0.05,0.00,0.00,0.00,0.00,0.05,0.05
532,Anecdotes litteraires1 1756,Raynal,9,9__Raynal_Anecdotes litteraires1 1756,4.55,2.05,1.10,0.40,1.90,1.25,...,0.20,0.10,0.00,0.00,0.00,0.00,0.00,0.05,0.15,0.05
533,Anecdotes litteraires2 1756,Raynal,9,9__Raynal_Anecdotes litteraires2 1756,3.85,1.15,1.30,0.85,1.45,1.10,...,0.20,0.05,0.00,0.00,0.05,0.15,0.05,0.00,0.15,0.05
534,ecole 2,Raynal,9,9__Raynal_ecole 2,2.90,0.90,0.85,0.55,1.70,0.45,...,0.00,0.05,0.05,0.10,0.10,0.00,0.00,0.00,0.05,0.05


In [10]:
raw_df = corpus.iloc[:, :4] # leaves metadata
X = corpus.iloc[:, 4:] # leaves only word columns

raw_df

,work,author,chunk_num,tag
0,Eclaircissemens,Baudeau,0,0__Baudeau_Eclaircissemens
1,Explication,Baudeau,0,0__Baudeau_Explication
2,Idees sur les besoins 2,Baudeau,0,0__Baudeau_Idees sur les besoins 2
3,Lettres a un magistraTtxt,Baudeau,0,0__Baudeau_Lettres a un magistraTtxt
4,Resultats,Baudeau,0,0__Baudeau_Resultats
...,...,...,...,...
531,Anecdotes historiques 1,Raynal,9,9__Raynal_Anecdotes historiques 1
532,Anecdotes litteraires1 1756,Raynal,9,9__Raynal_Anecdotes litteraires1 1756
533,Anecdotes litteraires2 1756,Raynal,9,9__Raynal_Anecdotes litteraires2 1756
534,ecole 2,Raynal,9,9__Raynal_ecole 2


In [11]:
# create numeric author labels
labels, label_uniques = raw_df.author.factorize()
raw_df.insert(1, "author_label", labels)
raw_df

,work,author_label,author,chunk_num,tag
0,Eclaircissemens,0,Baudeau,0,0__Baudeau_Eclaircissemens
1,Explication,0,Baudeau,0,0__Baudeau_Explication
2,Idees sur les besoins 2,0,Baudeau,0,0__Baudeau_Idees sur les besoins 2
3,Lettres a un magistraTtxt,0,Baudeau,0,0__Baudeau_Lettres a un magistraTtxt
4,Resultats,0,Baudeau,0,0__Baudeau_Resultats
...,...,...,...,...,...
531,Anecdotes historiques 1,4,Raynal,9,9__Raynal_Anecdotes historiques 1
532,Anecdotes litteraires1 1756,4,Raynal,9,9__Raynal_Anecdotes litteraires1 1756
533,Anecdotes litteraires2 1756,4,Raynal,9,9__Raynal_Anecdotes litteraires2 1756
534,ecole 2,4,Raynal,9,9__Raynal_ecole 2


In [12]:
# check unique authors
label_uniques.values

array(['Baudeau', 'Condorcet', 'dHolbach', 'Diderot', 'Raynal'],
      dtype=object)

In [13]:
print('N unique works:', len(list(set(raw_df.work))))

unique_works = list(set(raw_df.work))

# test
for w in unique_works[0:3]:
    print(w)

N unique works: 25
Pensees philosophiques
Explication
2


## NB: CHANGE PATH

In [14]:
# testing each individual work
for w in unique_works:
    
    problems = raw_df[raw_df.work == w].reset_index(drop=True).copy()
    problems_X = X[raw_df.work == w].reset_index(drop=True).copy()
    #print(problems)
    
    # extract true author
    a_true = problems.author[0]
    # print(a_true)
    
    # extract the rest of the corpus
    rest = raw_df[raw_df.work != w].reset_index(drop=True).copy()
    rest_X = X[raw_df.work != w].reset_index(drop = True).copy()
    #print(rest)
    
    
    print('\n\n########################## Working with:', w, '---- by', a_true)
    
    ### Scaling

    sts = StandardScaler(with_mean=False).fit(rest_X)
    rest_scaled_X = sts.transform(rest_X)
    problems_scaled_X = sts.transform(problems_X)


    # set verifier
    rng = np.random.default_rng(42)

    bdi_mm = BDIVerifier(
        metric='minmax', nb_bootstrap_iter=1000, rnd_prop=0.35, random_state=rng
    )

    # fit 
    bdi_mm.fit(rest_scaled_X, rest.author_label)
    
    for label in label_uniques.values:
        print(f"Testing against {label}")
        code = label_uniques.get_loc(label)
        print(
            f"Bootstrap Match Strength (one per chunk, 0-1.0): {bdi_mm.predict_proba(problems_scaled_X, [code] * problems_scaled_X.shape[0])}"
        )
    
    authors = label_uniques.values

    fh = ''
    
    # compare with each author and store results
    for a in authors:

        print(a)

        bdi_mm.predict_proba(
            problems_scaled_X, [label_uniques.get_loc(a)] * problems_scaled_X.shape[0]
        )


        x = pd.DataFrame(dict(zip(problems.tag, bdi_mm._dist_arrays)))

        fh = '03_tests/authors_vs_all/runs_5-authors/' + a_true + '-' + w + '_vs_' + a + '.csv'

        x.to_csv(fh)

03/19/2025 02:19:30 [ruzicka:INFO] Fitting on 530 documents...
03/19/2025 02:19:30 [ruzicka:INFO] Predicting on 6 documents




########################## Working with: Pensees philosophiques ---- by Diderot
Testing against Baudeau


03/19/2025 02:19:30 [ruzicka:INFO] Predicting on 6 documents


Bootstrap Match Strength (one per chunk, 0-1.0): [0.152 0.231 0.355 0.22  0.484 0.318]
Testing against Condorcet


03/19/2025 02:19:32 [ruzicka:INFO] Predicting on 6 documents


...

dHolbach


03/19/2025 02:33:13 [ruzicka:INFO] Predicting on 3 documents


Diderot


03/19/2025 02:33:14 [ruzicka:INFO] Predicting on 3 documents


Raynal
